In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
# green_path = '/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_green'
# orange_path = '/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_orange'
# purple_path = '/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_purple'
local_path = '/media/maryada/mdata/workspace/ssh_drive'

In [ ]:
fnames = glob.glob(f"{local_path}/NetworkActivity_EI_homeostasis_paradoxical.h5",recursive=True)
print(fnames)
# with pd.HDFStore(fnames[0]) as store:
#     df_network_activity = pd.DataFrame()
#     for key in store.keys():
#         iteration = key.split('_')[1]
#         repeat_count = key.split('_')[3]
#         df_net = store.get(key)

# Refactory period

In [ ]:
%config InlineBackend.figure_format='retina'
# subject_id = 2
dir_path = f'../onchip_experimentaldata/board_orange/2022-10-10/Refactory_period/16-15/'
data_path = f'{dir_path}/data'
img_path = f'{dir_path}/img'
netfnames = glob.glob(f"{data_path}/Network_*.h5",recursive=True)
with pd.HDFStore(netfnames[0]) as store:
    key = store.keys()[0]
    dfnetwork = store.get(key)


fnames = glob.glob(f"{data_path}/NeuronDynamics*.h5",recursive=True)
print(fnames)  
df_ISI = pd.DataFrame()
df_Spikes = pd.DataFrame()
with pd.HDFStore(fnames[0]) as store:
    for key in store.keys():
        df_data = store.get(key)

        dfisi = pd.DataFrame()
        fine_values = df_data.Fine.unique()
        for fine in fine_values:
            print(f'------------------------------{fine}------------------------------------')
            df = df_data[df_data.Fine == fine]
            active_neurons = df.Id.unique()
            for neuronId in active_neurons:
                Type = dfnetwork[dfnetwork.Id == neuronId].Type.values[0]
                spikes = df[df.Id == neuronId].sort_values('Timestamp').Timestamp.values            
                mean_isi =np.mean(np.diff(spikes))
                isi =np.diff(spikes)
                dict_isi = dict(NeuronId = len(isi)*[neuronId],  ISI = isi* 1e3, Type=len(isi)*[Type],Fine = fine,Coarse=df_data.coarse.unique()[0] )
                # print(dict_spikes)
                dfisi = pd.concat([dfisi,pd.DataFrame(dict_isi)],ignore_index=True)
            df_ISI = pd.concat([df_ISI,dfisi],ignore_index=True)

display(df_ISI)


# sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="bright",style='ticks',color_codes = True )
# figA = plt.figure(figsize=(20,10))

# sns.lineplot(data= df_FR,x="FineVal",y="fr",hue="Type",style="NeuronId")

# plt.show()#savefig(f"../data/board_orange/2022-08-07/RefP/14-37/reP.svg")
# plt.close(figA)

In [ ]:
figA = plt.figure(figsize=(10,6))

spike_time = df_data[(df_data.Id == 320) & (df_data.Fine == 10)].Timestamp.values * 1e3
print(np.diff(spike_time),spike_time)
plt.scatter(spike_time,len(spike_time) * [1], marker = '|')
plt.show()
plt.close(figA)

figA = plt.figure(figsize=(10,6))

spike_time = df_data[(df_data.Id == 303) & (df_data.Fine == 10)].Timestamp.values * 1e3
plt.scatter(spike_time,len(spike_time) * [1], marker = '|')
plt.show()
plt.close(figA)



In [ ]:
sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="bright",style='ticks',color_codes = True )
# figA = plt.figure(figsize=(20,20))

g = sns.FacetGrid(df_ISI_C2sm[df_ISI_C2sm.Fine<110], row='Type',col='Fine',hue='Type', margin_titles=True,sharex=False,xlim=(0,6))
g.map_dataframe(sns.histplot, x="ISI",binwidth=0.05,stat='percent')#, y="tip")
# plt.xlim(0,100)
plt.show()#savefig(f"../data/board_orange/2022-08-07/RefP/14-37/reP.svg")
# plt.close(figA)
plt.close()

# Firing rate

In [ ]:

# netfnames = glob.glob(f"{data_path}/Network_*.h5",recursive=True)
# with pd.HDFStore(netfnames[0]) as store:
#     key = store.keys()[0]
#     dfnetwork = store.get(key)

data_path =f'{orange_path}/2023-03-20/EI_homeostasis_MEMORY/12-47/data'
fnames = glob.glob(f"{data_path}/NetworkActivity*.h5",recursive=True)
print(fnames)  

with pd.HDFStore(fnames[0]) as store:
    for key in store.keys():
        iteration = int(key.replace('/','').split('_')[1])
        repeat = int(key.replace('/','').split('_')[-1])
        print(iteration,repeat)
        df = store.get(key).reset_index()       
        df_fr = pd.DataFrame()
        active_neurons = df.Id.unique()
        # print(active_neurons)
        for neuronId in active_neurons:
            spikes = df[df.Id == neuronId].sort_values('Timestamp').Timestamp.values
            Type = df[df.Id == neuronId].Type.unique()[0]
            duration = max(spikes)-min(spikes)
            dict_fr = dict( NeuronId = neuronId, fr = len(spikes),duration = duration, Repeat = repeat, Type=Type, Iteration = iteration)
            df_fr = pd.concat([df_fr,pd.DataFrame(dict_fr,index=[0])],ignore_index=True)
        df_fr.to_hdf(f'{data_path}/DC_FrequencyNeuron_wise.h5',append=True,key=f'Neuron_wise')


In [ ]:
df_dur = pd.read_hdf(f'{data_path}/DC_FrequencyNeuron_wise.h5',key=f'Neuron_wise')
df_dur.sort_values([''])

In [ ]:
%matplotlib widget
plt.ioff()

figA = plt.figure(figsize=(15,10))

df_dur = df_dur.astype({"Iteration": int, "Repeat": int, "duration": float, "fr": float, "NeuronId": int})
df_dur.dtypes
df = df_dur[ df_dur.Type == 'Pyr']
df = df.groupby(['Repeat','Iteration']).describe()['duration'][['max']].reset_index()
sns.scatterplot(data= df,x ='Iteration',y='max',s=5,edgecolor=None)
plt.show()
plt.close(figA)
plt.ion()



In [ ]:
df_dur = pd.read_hdf(f'{data_path}/DC_FrequencyNeuron_wise.h5',key=f'Neuron_wise')
# g = sns.FacetGrid(data=df_dur[df_dur.Type == 'Pyr'],height=6,aspect=1.5, col = 'Repeat',col_wrap=2)
# g.map_dataframe(sns.scatterplot,x='Iteration',y = 'duration')
df_dur.Iteration.unique()

In [ ]:
df_FR = df_FR.astype({"Input": int}).sort_values(by=['Input'])

sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="bright",style='ticks')
figA = plt.figure(figsize=(20,10))

sns.lineplot(data= df_FR,x="Input",y="fr",hue="Type",err_style="bars")

plt.savefig(f"{img_path}/DC-Freq.svg")
plt.close(figA)

# Interspike-interval

In [ ]:
%config InlineBackend.figure_format='retina'
# subject_id = 2
dir_path = f'{orange_path}/2022-10-10/Refactory_period/16-15/'
data_path = f'{dir_path}/data'
img_path = f'{dir_path}/img'
netfnames = glob.glob(f"{data_path}/Network_*.h5",recursive=True)
with pd.HDFStore(netfnames[0]) as store:
    key = store.keys()[0]
    dfnetwork = store.get(key)

fnames = glob.glob(f"{data_path}/NeuronDynamics*.h5",recursive=True)
print(fnames)  
df_ISI = pd.DataFrame()
df_Spikes = pd.DataFrame()
with pd.HDFStore(fnames[0]) as store:
    for key in store.keys():
        df_data = store.get(key)
        print(df_data.head())
        dfisi = pd.DataFrame()
        fine_values = df_data.Fine.unique()
        for fine in fine_values:
            print(f'------------------------------{fine}------------------------------------')
            df = df_data[df_data.Fine == fine]
            active_neurons = df.Id.unique()
            for neuronId in active_neurons:
                Type = dfnetwork[dfnetwork.Id == neuronId].Type.values[0]
                spikes = df[df.Id == neuronId].sort_values('Timestamp').Timestamp.values            
                mean_isi =np.mean(np.diff(spikes))
                isi =np.diff(spikes)
                dict_isi = dict(NeuronId = len(isi)*[neuronId],  ISI = isi* 1e3, Type=len(isi)*[Type],Fine = fine,Coarse=df_data.coarse.unique()[0] )
                # print(dict_spikes)
                dfisi = pd.concat([dfisi,pd.DataFrame(dict_isi)],ignore_index=True)
            df_ISI = pd.concat([df_ISI,dfisi],ignore_index=True)

display(df_ISI)


# sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="bright",style='ticks',color_codes = True )
# figA = plt.figure(figsize=(20,10))

# sns.lineplot(data= df_FR,x="FineVal",y="fr",hue="Type",style="NeuronId")

# plt.show()#savefig(f"../data/board_orange/2022-08-07/RefP/14-37/reP.svg")
# plt.close(figA)

In [ ]:
dfnetwork

In [ ]:
sns.histplot(df_ISI.reset_index(drop=True), x="ISI", binwidth=0.05,stat='percent').set(xlim=(0,10))


# ax2 = ax_hist.inset_axes([0.3, .5, 0.2, .2], facecolor='lightgrey')
# sns.histplot(isi_DF.reset_index(drop=True), x="inter_spike", binwidth=.05,stat='percent',ax=ax2)
# ax2.set_title('Zoom in')
# ax2.set_xlabel('Time (msec)', fontsize=10)
# ax2.set_ylabel('Percent (%)', fontsize=10)
# ax2.set_xlim([isi_DF.inter_spike.min()-0.1,isi_DF.inter_spike.max()+0.1])

In [ ]:
%config InlineBackend.figure_format='retina'
# subject_id = 2
data_path = "../onchip_experimentaldata/board_orange/2022-10-11/DC_FF_curve/15-46/data"
netfnames = glob.glob(f"{data_path}/Network_DC_*.h5",recursive=True)
with pd.HDFStore(netfnames[0]) as store:
    key = store.keys()[0]
    dfnetwork = store.get(key)


fnames = glob.glob(f"{data_path}/NetworkActivity_DC_*.h5",recursive=True)
print(fnames)  
df_ISI = pd.DataFrame()
df_Spikes = pd.DataFrame()
df_FR = pd.DataFrame()
with pd.HDFStore(fnames[0]) as store:
    for key in store.keys():
        trial = 0#key.replace('/','').split('_')[1]
        iteration = 0#key.replace('/','').split('_')[-1]
        df = store.get(key)
        
        dfisi = pd.DataFrame()
        dfspikes = pd.DataFrame()
        df_fr = pd.DataFrame()
        active_neurons = df.Id.unique()
        print(active_neurons)
        for neuronId in active_neurons:
            
            Type = dfnetwork[dfnetwork.Id == neuronId].Type.values[0]

            spikes = df[df.Id == neuronId].sort_values('Timestamp').Timestamp.values
            
            dict_fr = dict(NeuronId = neuronId,  fr = len(spikes), Iteration = iteration,Type=Type, Trial = trial)
            df_fr = pd.concat([df_fr,pd.DataFrame(dict_fr,index=[0])],ignore_index=True)
            df_FR = pd.concat([df_FR,df_fr],ignore_index=True)

            # mean_isi =np.mean(np.diff(spikes))
            # isi =np.diff(spikes)
            # dict_isi = dict(NeuronId = len(isi)*[neuronId],  ISI = isi, Iteration = len(isi)*[iteration],Type=len(isi)*[Type], Trial = len(isi)*[trial])
            dict_spikes = dict(NeuronId = len(spikes)*[neuronId],  spikes = spikes, Iteration = len(spikes)*[iteration],Type=len(spikes)*[Type], Trial = len(spikes)*[trial])
            # print(dict_spikes)
            # dfisi = pd.concat([dfisi,pd.DataFrame(dict_isi)],ignore_index=True)
            dfspikes = pd.concat([dfspikes,pd.DataFrame(dict_spikes)],ignore_index=True)
        # df_ISI = pd.concat([df_ISI,dfisi],ignore_index=True)
        df_Spikes = pd.concat([df_Spikes,dfspikes],ignore_index=True)
# df_ISI = df_ISI.astype({"Iteration": int})
# df_ISI
# df_Spikes = df_Spikes.astype({"Iteration": int, "Trial": int})
df_FR #= df_FR.astype({"FineVal": int})

# df_Spikes

# sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="bright",style='ticks',color_codes = True )
# figA = plt.figure(figsize=(20,10))

# sns.lineplot(data= df_FR,x="FineVal",y="fr",hue="Type",style="NeuronId")

# plt.show()#savefig(f"../data/board_orange/2022-08-07/RefP/14-37/reP.svg")
# plt.close(figA)

# DC-Freq curve

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os
%config InlineBackend.figure_format='retina'
# subject_id = 2
dir_path = f"{green_path}/2023-04-01/IF_curve/16-41"
data_path =f"{dir_path}/data"
img_path =f"{dir_path}/img"
with pd.HDFStore(f'{data_path}/DC_Frequency.h5') as store:
    print(store.keys())
    key = store.keys()[0]
    df_IF = store.get(key)

sns.lineplot(data=df_IF.reset_index(drop=True),x='Input',y='FiringRate',hue='Type',palette='Set1')#.set(xlim=(0,50))

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os
%config InlineBackend.figure_format='retina'
# subject_id = 2
dir_path = "/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_orange/2022-11-04/EI_persistancy/21-43"
data_path =f"{dir_path}/data"
img_path =f"{dir_path}/img"
with pd.HDFStore(f'{data_path}/DC_FrequencyNeuron_wise.h5') as store:
    key = store.keys()
    print(key)

# sns.lineplot(data=df_IF,x='Input',y='FiringRate',hue='Type')#.set(xlim=(0,50))

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os
%config InlineBackend.figure_format='retina'
# subject_id = 2
dir_path = "/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_green/2022-11-08/EI_homeostasis_paradoxical/15-09"

data_path =f"{dir_path}/data"
img_path =f"{dir_path}/img"
df_firingRate =pd.DataFrame()
with pd.HDFStore(f'{data_path}/NetworkActivity_EI_homeostasis_paradoxical.h5') as store:
    key = store.keys()[0]
    # print(store.keys())
    df_firingRate = store.get(key).reset_index()

df_firingRate

In [ ]:
df_Pyr = df_firingRate[ (df_firingRate.Type == 'Pyr') ]
df_PV = df_firingRate[ (df_firingRate.Type == 'PV') ]

bins = np.arange(0.,1.,0.05)
def get_firing_rate(ti):
    #IMPORTANT: CHECK UNITS
    hists,_ =np.histogram(ti,bins=bins)
    fireRate= np.zeros(len(bins))
    idx_count=1
    for n in hists:
        fireRate[idx_count] = n / 0.05 
        idx_count+=1
    # print(fireRate)
    return np.mean(fireRate)

df_fr_Pyr = df_Pyr.groupby(['Id','Repeat'])['Timestamp'].agg(get_firing_rate)
df_fr_Pyr
df_fr_PV = df_PV.groupby(['Id','Repeat'])['Timestamp'].agg(get_firing_rate)


sns.lineplot(data = df_fr_Pyr.reset_index(),x='Repeat',y='Timestamp',hue='Id',palette='Reds',legend=False)

# g = sns.lineplot(data = df_fr_PV.reset_index(),x='Repeat',y='Timestamp',hue='Id',palette='Blues',legend=False)
# g.set(xlabel ='Input (a.u.)',ylabel='Firing rate (Hz)')
# plt.show()
# plt.close()

In [ ]:
plt.ioff()

figA = plt.figure(figsize=(6,6),dpi=100)
sns.lineplot(data= df,x="Input",y="FiringRate",style="Repeat",hue="Type").set(xlabel='DC Coarse 2', ylabel = 'Firing rate in Hz')#,xlim=(0,250),ylim=(0,250))
# plt.axis('equal')
# plt.savefig(f"{img_path}/Input_Frequency.svg",facecolor = 'white')
plt.show()

plt.close(figA)
plt.ion()

# Raster plots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
%config InlineBackend.figure_format='retina'

# dir_path = f'{orange_path}/2023-03-20/EI_homeostasis_MEMORY/12-47'
# data_path = f'{dir_path}/data'
# #/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_green/2023-03-04/EI_homeostasis/19-02
# img_path = f"{dir_path}/img"

fnames = glob.glob(f"{data_path}/NetworkActivity_EI*.h5",recursive=True)

with pd.HDFStore(fnames[0]) as store:
    df_network_activity = pd.DataFrame()
    for key in store.keys():
        iteration = key.split('_')[1]
        repeat_count = key.split('_')[3]
        df_net = store.get(key)
        df_net['Iteration'] = int(iteration)
        df_net['Repeat'] = int(repeat_count)
        df_net = df_net.reset_index()
        df_network_activity = pd.concat([df_network_activity,df_net],ignore_index=True)

df_network_activity

In [ ]:
df_network_activity.Iteration.unique()

In [ ]:
import itertools

sns.set_context('notebook')
os.makedirs(f"{img_path}/analysis_plots",exist_ok=True)
pd.options.mode.chained_assignment = None 
# for iteration,repeat in itertools.product([20,100,200,300,399],range(5)):
for iteration in [20,30,50,80,100,200,300,399]:
    fig_rater = plt.figure(figsize=(16,6),dpi=100)

    print(iteration,repeat)
    df_spike_raster= df_network_activity[(df_network_activity.Iteration == iteration) & (df_network_activity.Repeat == 2)]
    print(df_spike_raster.head())
    # df_spike_raster['time'] = df_spike_raster.Timestamp + repeat
    df_spike_raster.loc[:,'time'] = df_spike_raster.apply(lambda row: row.Timestamp + repeat, axis=1)
    sns.scatterplot(data=df_spike_raster,x='time',y='Id',hue='Type',palette='Set1', s = 3, edgecolor= None,legend=False)#.set(ylim = (0,400))
    plt.title(f'Iteration {iteration}')
    # plt.savefig(f'{img_path}/analysis_plots/raster_Iteration_{iteration}.svg')
    plt.show()
    plt.close()

In [ ]:
import itertools

sns.set_context('notebook')
os.makedirs(f"{img_path}/analysis_plots",exist_ok=True)
pd.options.mode.chained_assignment = None 
# for iteration,repeat in itertools.product([20,100,200,300,399],range(5)):
for iteration in [20,100,200,300,399]:
    fig_rater = plt.figure(figsize=(16,6))

    for repeat in range(5):
        print(iteration,repeat)
        df_spike_raster= df_network_activity[(df_network_activity.Iteration == iteration) & (df_network_activity.Repeat == repeat)]
        print(df_spike_raster.head())
        # df_spike_raster['time'] = df_spike_raster.Timestamp + repeat
        df_spike_raster.loc[:,'time'] = df_spike_raster.apply(lambda row: row.Timestamp + repeat, axis=1)
        sns.scatterplot(data=df_spike_raster,x='time',y='Id',hue='Type',palette='Set1', s = 1, edgecolor= None,legend=False).set(ylim = (0,400))
    plt.title(f'Iteration {iteration}')
    plt.savefig(f'{img_path}/analysis_plots/raster_Iteration_{iteration}.svg')
    plt.show()
    plt.close()

    # df_Pyr = df_spike_raster[(df_spike_raster.Type.str.contains('Pyr'))]
    # df_PV = df_spike_raster[ (df_spike_raster.Type.str.contains('PV')) ]
    # print(df_Pyr.Timestamp.max())
    # def get_firing_rate(ti,dt):
    #     #IMPORTANT: CHECK UNITS
    #     bins = np.arange(0.,1.,dt)
    #     hists,_ =np.histogram(ti,bins=bins)
    #     fireRate= np.zeros(len(bins))
    #     idx_count=1
    #     for n in hists:
    #         fireRate[idx_count] = n/dt
    #         idx_count+=1
    #     df_temp = pd.DataFrame(dict(bins = np.around(bins,3), Avg_firing_rate = fireRate))
    #     return df_temp

    # df_P = df_Pyr.groupby(['Id']).apply(lambda x: get_firing_rate(x.Timestamp,0.02),)#.rename('Avg_firing_rate').reset_index().explode('Avg_firing_rate')
    # df_Pyr_neuron = df_P.reset_index().drop('level_1',axis=1)

    # df_Pyr_neuron_hist = pd.pivot(df_Pyr_neuron, columns='bins', index='Id', values='Avg_firing_rate')
    # fig_rater = plt.figure(figsize=(4,4))

    # sns.heatmap(df_Pyr_neuron_hist, cmap='coolwarm',vmin=0, vmax=250,cbar_kws={'label': 'Firing rate (Hz)'})
    # plt.savefig(f'{img_path}/analysis_plots/heatmap_{iteration}.svg',transparent=False)

    # plt.show()
    # plt.close()

In [ ]:
# avg_E = []
# for nid in df_Pyr.Id.unique():
#     df_Pyr_n  = df_Pyr[df_Pyr.Id==nid]
#     avg_E.append(len(df_Pyr_n.Timestamp)/max(df_Pyr_n.Timestamp))
df_spike_raster

In [ ]:
import itertools
import numpy as np
bins = np.arange(0.,1.03,0.03)

def GetFiringFrequency(ti,N,bins,t_window=.03):
    #IMPORTANT: CHECK UNITS
    hists,_ =np.histogram(ti,bins=bins)
    fireRate= np.zeros(len(bins))
    idx_count=1
    for n in hists:
        fireRate[idx_count] = n / (t_window * N)
        idx_count+=1
    return fireRate,bins
# incorrectT = [5,8,14,16,19]
incorrectT = [2,4,6]#,16,19]
def plot_firingrates(df_network_activity):
    ''' raster plot '''
    df_FR = pd.DataFrame()
    for iteration,repeat in itertools.product(df_network_activity.Iteration.unique(),df_network_activity.Repeat.unique()):
        # print(iteration,repeat)
        if repeat not in incorrectT:
            continue
        # f = plt.figure(figsize=(20,20),dpi=100)
        df_spikes_PV = df_network_activity.query("Type =='PV' and Repeat==@repeat and Iteration==@iteration")
        df_spikes_Pyr = df_network_activity.query("Type =='Pyr' and Repeat==@repeat and Iteration==@iteration")
        df_spikes_Inp = df_network_activity.query("Type =='Input' and Repeat==@repeat and Iteration==@iteration")
        fireRate_PV,tbins_PV = GetFiringFrequency(df_spikes_PV.Timestamp.values,len(df_spikes_PV.Id.unique()),bins)
        df_PV = pd.DataFrame(list(zip(fireRate_PV,tbins_PV)),columns=['Firing_rate','Time_bin'])
        df_PV['Repeat'] = repeat
        df_PV['Type'] = 'PV'
        fireRate_Pyr,tbins_Pyr = GetFiringFrequency(df_spikes_Pyr.Timestamp.values,len(df_spikes_Pyr.Id.unique()),bins)
        df_Pyr = pd.DataFrame(list(zip(fireRate_Pyr,tbins_Pyr)),columns=['Firing_rate','Time_bin'])
        df_Pyr['Repeat'] = repeat
        df_Pyr['Type'] = 'Pyr'
        
        # fireRate_Inp,tbins_Inp = GetFiringFrequency(df_spikes_Inp.Timestamp.values,len(df_spikes_Inp.Id.unique()),bins)
        # df_Inp = pd.DataFrame(list(zip(fireRate_Inp,tbins_Inp)),columns=['Firing_rate','Time_bin'])
        # df_Inp['Repeat'] = repeat
        # df_Inp['Type'] = 'Input'
        df_FR = pd.concat([df_FR,df_PV],ignore_index=True)
        # fig_fr = plt.figure(figsize=(14,8))
        # plt.plot(tbins_PV,fireRate_PV,color = 'b')
        # plt.plot(tbins_Pyr,fireRate_Pyr,color = 'r')
        # plt.axvline(0.5,0,50)
        # plt.axvline(0.5+.25,0,50)
        # plt.show()
        # plt.close()
    return df_FR


df_FR = plot_firingrates(df_network_activity.reset_index())
df_FR

# kwrg = dict(marker = '.', s = 10)
# g = sns.FacetGrid(df_FR, height=3,aspect=1.5, margin_titles=True,sharex= False,sharey= True)
plt.figure(figsize=(10,5))
g = sns.lineplot(data=df_FR, x="Time_bin", y="Firing_rate",hue = "Type",
palette=['#000000',"#FF0B04", "#4374B3"],hue_order=['Input','PV','Pyr'])
g.set(xlabel = 'Time (sec)',ylabel ='Firing Rate (Hz)')
# g.fig.subplots_adjust(top=.96) 
# g.fig.suptitle(f'ITERATION :: {iteration}')
# g.add_legend()
# g.set_xlabels('Time (sec)')
# g.set_ylabels('Firing Rate (Hz)')
# print(g.axes_dict.items())
plt.axvline(x=0.3,ymin=0,ymax=50)
plt.axvline(x=0.3+.4,ymin=0,ymax=50)
plt.legend(loc="lower right")#, bbox_to_anchor=(.02, .01))
plt.tight_layout()
# for col_val, ax in g.axes_dict.items():
#     ax.axvspan(xmin=0.3, xmax=0.3 +.15, facecolor='yellow', alpha=0.3)
plt.show()#savefig(f'{img_path}/paradoxical.svg',facecolor='white', transparent=False)
plt.close()


In [ ]:
def plot_events(df_network_activity,img_path="",imgfname="raster"):
    ''' raster plot '''
    colors = ['#000000',"#FF0B04", "#4374B3"]# Set your custom color palette
    sns.set_palette(sns.color_palette(colors))

    for iteration in df_network_activity.Iteration.unique():
        # f = plt.figure(figsize=(20,20),dpi=100)
        kwrg = dict(marker = '.', s = 10)
        g = sns.FacetGrid(df_network_activity[df_network_activity.Iteration==iteration ], col="Repeat",col_wrap=5,height=3,aspect=1.5, margin_titles=True,sharex= False,sharey= True)

        g.map_dataframe(sns.scatterplot, x="Timestamp", y="Id",hue = "Type",**kwrg)
        g.fig.subplots_adjust(top=.96) 
        g.fig.suptitle(f'ITERATION :: {iteration}')
        g.add_legend()
        g.set_xlabels('Time (sec)')
        g.set_ylabels('Neuron Id')
        # print(g.axes_dict.items())
        sns.move_legend(g, "upper right")#, bbox_to_anchor=(.02, .01))
        g.tight_layout()
        for col_val, ax in g.axes_dict.items():
            ax.axvspan(xmin=0, xmax=0.03, facecolor='yellow', alpha=0.3)
        plt.savefig(f'{img_path}/raster_plots/{imgfname}_{iteration}.png',facecolor='white', transparent=False)
        plt.close()

# CV

In [ ]:
df_CV = pd.DataFrame()

for iteration in df_network_activity.Iteration.unique():
    for repeat in df_network_activity.Repeat.unique():
    
        df_spikes = df_network_activity.query("Type =='Pyr' and Repeat==@repeat and Iteration==@iteration")
        # figA= plt.figure(figsize=(15,5))
        for j in df_spikes.Id.unique():
            spike_B = df_spikes.query(f'Id == @j ').Timestamp.values
            if len(spike_B) >1: 
                CV = coefficient_variation(spike_B,square=True)
                df_CV = pd.concat([df_CV,pd.DataFrame({'Id' : j,'CV': CV,'Repeat' : repeat,'Iteration': iteration,'Type' : 'Pyr'},index=[0])])


In [ ]:
df_CV.to_hdf(f'{data_path}/CV.h5',append=True,key='Pyr')


In [ ]:
dir_path = "/media/mb/Data/DYNAP/EI_homeostasis/data/onchip_experimentaldata/board_orange/2022-10-17/EI_persistancy_SHUNT/16-49"
data_path = f"{dir_path}/data"
img_path = f"{dir_path}/img"
Type = 'Pyr'
# df_CV = pd.read_hdf(f'{data_path}/CV.h5',key=Type).reset_index(drop=True)


In [ ]:
for repeat in range(4):

    g = sns.jointplot(data=df_CV[(df_CV.Repeat==repeat)], x="Iteration",color='b', y="CV",s=3)
    g.fig.set_figwidth(18.50)
    g.ax_marg_x.remove()
    plt.show()
    plt.close()
    # figA = plt.figure(figsize=(11,6),dpi=100)

    # g = sns.pointplot(data=df_CV[(df_CV.Repeat==repeat)], x="Iteration",color='k',errwidth=.4, y="CV",markers='.',linestyles='--')
    # plt.xticks(df_CV.Iteration.unique()[::20])

    # plt.show()
    # plt.close()

# CC

In [ ]:

for iteration in df_network_activity.Iteration.unique()[:20]:
    print(f'--------------{iteration}------------------')
    df_CC = pd.DataFrame()
    for repeat in df_network_activity.Repeat.unique():
    
        df_spikes = df_network_activity.query("Type =='Pyr' and Repeat==@repeat and Iteration==@iteration")
        # figA= plt.figure(figsize=(15,5))
        for i in df_spikes.Id.unique():
            spike_A = df_spikes.query(f'Id == @i').Timestamp.values
            CC = []
            for j in df_spikes.Id.unique():
                spike_B = df_spikes.query(f'Id == @j').Timestamp.values
                if i !=j and len(spike_B) > 2 and len(spike_A) > 2:
                    temp_CC = correlation_coefficient(spike_A,spike_B)[0][1]              
                    CC += [temp_CC]
            if len(CC) > 0:
                mean_CC = np.mean(CC)
            else:
                mean_CC = 0
            
            df_CC = pd.concat([df_CC,pd.DataFrame({'Id' : i,'CC': mean_CC,'Repeat' : repeat,'Iteration': iteration,'Type' : 'Pyr'},index=[0])])
    df_CC.to_hdf(f'{data_path}/CC.h5',append=True,key='Pyr')

In [ ]:
sns.set_theme("talk", font_scale=1.2, rc={"lines.linewidth": 2.5},style='ticks' )
sns.set_palette("YlOrBr")
%matplotlib inline
# plt.ioff()
figC = plt.figure(figsize=(15,15)) 
g = sns.FacetGrid(df_CC, col="Repeat",col_wrap=2,height=3,aspect=1,margin_titles=True,sharex= False,sharey= True)

g.map_dataframe(sns.scatterplot, y="CC", x="Id",s=5,marker='s',color='k',edgecolor=None)
g.add_legend()
g.tight_layout()
plt.show()
plt.close(figC)
# plt.ion()


# Convergence Trend

In [ ]:
from pathlib import Path
path = Path("'/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_orange/2023-01-19/EI_homeostasis/12-09/data/firing_rate.h5'")
print(path.parents[0])

 For all seeds and both boards, highlight the target
 


In [ ]:
%config InlineBackend.figure_format='retina'
# subject_id = 2
import time
from pathlib import Path
import matplotlib.patches as mpatches

E_TARGET = 40
I_TARGET = E_TARGET * 2
#/media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_green/
# dir_path = f"{green_path}/2023-04-16/EI_homeostasis_static_inh/*" 
dir_path = f"{orange_path}/EI_homeostasis_MC/15-39" 
# /media/mb/Data/DYNAP/EI_homeostasis/results/onchip_experimentaldata/board_orange/EI_homeostasis_MC/15-39/data
data_path =f"{dir_path}/data"

fnames = glob.glob(f"{data_path}/firing_rate.h5",recursive=True)
print(fnames)
n_counter  = 10
for fname in fnames:
    print(fname)

    path = Path(f"{fname}")
    img_path =f"{path.parents[1]}/img"
    os.makedirs(img_path,exist_ok=True)

    with pd.HDFStore(fname) as store:
        for key in store.keys():
            df_firingRate = store.get(key)
    display(df_firingRate.head())
    # # df_firingRate = df_firingRate[df_firingRate.Repeat_count>0]
    sns.set_theme("talk", font_scale=1., rc={"lines.linewidth": 2.5}, palette="Spectral_r",style='ticks',color_codes = True )
    # # figA = plt.figure(figsize=(10,10))

    g = sns.FacetGrid(df_firingRate, height=5,aspect=1.5, margin_titles=True,sharex=False)
    # g.map_dataframe(sns.scatterplot, x="Iteration", y="AvgE",s=5,color='r',edgecolor=None)
    # g.map_dataframe(sns.scatterplot, x="Iteration", y="AvgI",s=5,color='b',edgecolor=None)
    g.map_dataframe(sns.scatterplot, x="Iteration", y="AvgE",hue='Repeat_count',s=5,palette='Blues',edgecolor=None)
    g.map_dataframe(sns.scatterplot, x="Iteration", y="AvgI",hue='Repeat_count',s=5,palette='Reds',edgecolor=None)
    # # plt.title('Test network with converged weights')
    g.set_ylabels('Pop. firing rate (Hz)')
    g.set_xlabels('Iterations')
    g.refline(y=I_TARGET,
            color = "black",
            lw = 1)
    g.refline(y = E_TARGET,
            color = "black",
            lw = 1)
    red_patch = mpatches.Patch(color='#cc4125', label='Parvalbumin (PV)')
    blue_patch = mpatches.Patch(color='#3d85c6', label='Pyramidal (Pyr)')
    plt.legend(title='5 Clusters',loc='lower right',handles=[red_patch,blue_patch])
#     plt.ylim(0,100)
    plt.savefig(f"{img_path}/convergence_trend.svg",transparent=True)
    plt.show()

    # # plt.savefig(f"{img_path}/convergence_trend_zoomed.png")
    plt.close()

    wfnames = glob.glob(f"{path.parents[1]}/data/weight_trend.h5",recursive=True)

    with pd.HDFStore(wfnames[0]) as store:
        for key in store.keys():
            df_weight = store.get(key)
    # display(df_weight[(df_weight.source=='Pyr')&(df_weight.target == 'PV')])
    sns.set_theme("talk", font_scale=1.1, rc={"lines.linewidth": 2.5}, palette="Set1",style='ticks',color_codes = True )
    figA = plt.figure(figsize=(6,6))

    g = sns.FacetGrid(df_weight, col="source",hue='coarse',height=3,aspect=1,row='target', margin_titles=True,sharex=False)
    g.map_dataframe(sns.scatterplot, x="Iteration", y="fine",s=8,edgecolor=None)

    g.add_legend()
    plt.savefig(f"{img_path}/weight_trend.svg")
    plt.show()
    plt.close(figA)

In [ ]:
fnames = glob.glob(f"{data_path}/NetworkActivity_*.h5",recursive=True)

df_network_activity = pd.DataFrame()
with pd.HDFStore(fnames[0]) as store:
    for key in store.keys():
        print(key)
        df_network_activity = store.get(key)


sns.set_context('notebook')
os.makedirs(f"{img_path}/analysis_plots",exist_ok=True)
pd.options.mode.chained_assignment = None 
# for iteration,repeat in itertools.product([20,100,200,300,399],range(5)):
for iteration in [20,100,200,300,399]:

    for repeat in range(5):
        print(iteration,repeat)
        df_spike_raster= df_network_activity[(df_network_activity.Iteration == iteration) & (df_network_activity.Repeat == repeat)]
        # df_spike_raster['time'] = df_spike_raster.Timestamp + repeat
        df_spike_raster.loc[:,'time'] = df_spike_raster.apply(lambda row: row.Timestamp + repeat, axis=1)
        sns.scatterplot(data=df_spike_raster,x='time',y='Id',hue='Type',palette='Set1', s = 1, edgecolor= None,legend=False).set(ylim = (0,400))
    plt.title(f'Iteration {iteration}')
    plt.savefig(f'{img_path}/analysis_plots/raster_Iteration_{iteration}.svg')
    plt.show()
    plt.close()

    # df_Pyr = df_spike_raster[(df_spike_raster.Type.str.contains('Pyr'))]
    # df_PV = df_spike_raster[ (df_spike_raster.Type.str.contains('PV')) ]
    # print(df_Pyr.Timestamp.max())
    # def get_firing_rate(ti,dt):
    #     #IMPORTANT: CHECK UNITS
    #     bins = np.arange(0.,1.,dt)
    #     hists,_ =np.histogram(ti,bins=bins)
    #     fireRate= np.zeros(len(bins))
    #     idx_count=1
    #     for n in hists:
    #         fireRate[idx_count] = n/dt
    #         idx_count+=1
    #     df_temp = pd.DataFrame(dict(bins = np.around(bins,3), Avg_firing_rate = fireRate))
    #     return df_temp

    # df_P = df_Pyr.groupby(['Id']).apply(lambda x: get_firing_rate(x.Timestamp,0.02),)#.rename('Avg_firing_rate').reset_index().explode('Avg_firing_rate')
    # df_Pyr_neuron = df_P.reset_index().drop('level_1',axis=1)

    # df_Pyr_neuron_hist = pd.pivot(df_Pyr_neuron, columns='bins', index='Id', values='Avg_firing_rate')
    # fig_rater = plt.figure(figsize=(4,4))

    # sns.heatmap(df_Pyr_neuron_hist, cmap='coolwarm',vmin=0, vmax=250,cbar_kws={'label': 'Firing rate (Hz)'})
    # plt.savefig(f'{img_path}/analysis_plots/heatmap_{iteration}.svg',transparent=False)

    # plt.show()
    # plt.close()